In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense,Flatten
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder



In [19]:
# Load the dataset
data = pd.read_csv('/content/Kikuyu_Words.csv')  # Assuming the dataset is stored in 'french_pos_data.csv'

In [20]:
data.head(25)

,Word,Label
0,Mũndũ,Noun
1,Mũaki,Noun
2,Ihũa,Noun
3,Kĩrĩma,Noun
4,Gĩtĩri,Noun
5,Ikara,Verb
6,Rehe,Verb
7,Tengera,Verb
8,Rũga,Verb
9,koma,Verb


In [21]:
# Preprocessing
data["Word"] = data["Word"].str.replace("ũ", "u")
data["Word"] = data["Word"].str.replace("ĩ", "i")
data['Word'] = data['Word'].str.lower()

In [22]:
data.head(25)

,Word,Label
0,mundu,Noun
1,muaki,Noun
2,ihua,Noun
3,kirima,Noun
4,gitiri,Noun
5,ikara,Verb
6,rehe,Verb
7,tengera,Verb
8,ruga,Verb
9,koma,Verb


In [25]:
# Encoding labels
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

In [7]:
# Preprocessing
X = data['Word'].values
X

array(['mundu', 'muaki', 'ihua', 'kirima', 'gitiri', 'ikara', 'rehe',
       'tengera', 'ruga', 'koma', 'andika', 'tuma', 'cukuru', 'handu',
       'muteti', 'muiko', 'muitu', 'ndereba', 'murogi', 'kihii',
       'murutani', 'muiru', 'muici', 'kiratu', 'muruthi', 'thiia',
       'kimbu', 'nugu', 'kingangi', 'kahiu', 'nyungu', 'gakaraku',
       'mutune', 'mweru', 'muiru', 'njau', 'mbakuri', 'twara', 'roga',
       'tura', 'rima', 'enda', 'onja', 'aka', 'endia', 'toga', 'rwara',
       'ria', 'hokeka', 'uma', 'thoma', 'enyuka', 'ora', 'agana', 'raiha',
       'kiga', 'kura', 'mwihokeku', 'mwonju', 'muthomu', 'muumu',
       'nyenyuku', 'njuru', 'njaganu', 'ndaihu', 'ngigu', 'nguru',
       'inyui', 'ithui', 'nii', 'othee', 'wee', 'atia', 'riria', 'nuu',
       'ma', 'umuthi', 'niki', 'tene', 'riu', 'hwaiini'], dtype=object)

In [27]:
y = data['Label'].values
y

array([2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1])

In [29]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
# Tokenize words and pad sequences
word_to_index = {word: idx + 1 for idx, word in enumerate(set(X))}
X_train_tokens = np.array([[word_to_index[word] for word in sentence.split()] for sentence in X_train])
X_test_tokens = np.array([[word_to_index[word] for word in sentence.split()] for sentence in X_test])

max_sequence_length = max(max(len(x) for x in X_train_tokens), max(len(x) for x in X_test_tokens))
X_train_padded = pad_sequences(X_train_tokens, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_tokens, maxlen=max_sequence_length, padding='post')


In [31]:
# Convert labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

In [35]:
# Define and train the BiLSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_to_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(num_classes, activation='softmax'))


In [36]:
# model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1, 100)            8100      
                                                                 
 bidirectional_2 (Bidirecti  (None, 1, 256)            234496    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 1, 5)              1285      
                                                                 
Total params: 243881 (952.66 KB)
Trainable params: 243881 (952.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
# Defining loss function and the appropriate optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
# Train the model
model.fit(X_train_padded, y_train_one_hot,
          epochs=10, batch_size=32,
          validation_data=(X_test_padded,
                           y_test_one_hot))

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    